In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import librosa

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

from keras.models import Sequential
from keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import RMSprop
from keras.callbacks import ReduceLROnPlateau

In [ ]:
labels = pd.read_csv('labels.csv')
length = labels.shape[0]
idx = 0

while idx < length:
    filepath = labels.iloc[idx]['filepath']
    genre = labels.iloc[idx]['filename']

    y, sr = librosa.load(filepath)

    fig = make_subplots(rows=1, cols=2, subplot_titles=['Waveform', 'Chroma Features'])

    ax1 = fig.add_trace(go.Scatter(y=y, line=dict(color='blue')), row=1, col=1)

    chromagram = librosa.feature.chroma_stft(y=y, sr=sr)
    ax2 = fig.add_trace(go.Heatmap(z=chromagram, colorscale='Viridis'), row=1, col=2)

    fig.update_layout(height=600, width=1000, title=genre)
    fig.show()

In [ ]:
def plot_history(history):
    fig = make_subplots(rows=2, cols=1, subplot_titles=("Accuracy eval", "Error eval"))

    fig.add_trace(go.Scatter(y=history.history["accuracy"], mode='lines', name='Train Accuracy'), row=1, col=1)
    fig.add_trace(go.Scatter(y=history.history["val_accuracy"], mode='lines', name='Validation Accuracy'), row=1, col=1)

    fig.add_trace(go.Scatter(y=history.history["loss"], mode='lines', name='Train Error'), row=2, col=1)
    fig.add_trace(go.Scatter(y=history.history["val_loss"], mode='lines', name='Validation Error'), row=2, col=1)

    fig.update_yaxes(title_text="Accuracy", row=1, col=1)
    fig.update_xaxes(title_text="Epoch", row=1, col=1)
    fig.update_yaxes(title_text="Error", row=2, col=1)
    fig.update_xaxes(title_text="Epoch", row=2, col=1)

    fig.update_layout(height=800, width=800, title_text="Training History")
    fig.show()

In [ ]:
df = pd.read_csv('gtzan_features.csv')

X = df.iloc[:, :-1].values
y = df['labels'].values

X = X.reshape(X.shape[0], X.shape[1], 1, 1)

In [ ]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

X_train, X_temp, y_train, y_temp = train_test_split(X, y_encoded, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=X_train.shape[1:], padding='same'),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2), padding='same'),
    Flatten(),
    Dense (128, activation='relu'),
    Dropout(0.5),
    Dense(len(np.unique(y)), activation='softmax')
])
optimiser = RMSprop(learning_rate=0.0001)
model.compile(optimizer=optimiser,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)
history = model.fit(X_train, y_train, epochs=50,
                    validation_data=(X_val, y_val),
                    callbacks=[reduce_lr])

In [ ]:
y_pred = model.predict(X_test)
y_pred_decoded = np.argmax(y_pred, axis=1)
accuracy = accuracy_score(y_test, y_pred_decoded)
print(f'Test accuracy: {round(accuracy, 3)* 100}%')

In [ ]:
plot_history(history)